## 드롭아웃

- 과적합을 해결하는 방법으로 드롭아웃(dropout)이 있다.

- 신경망 모델이 과적합되는 것을 피하기 위한 방법으로, 학습 과정 중 임의로 일부 노드들을 학습에서 제외시킨다.

![Image](https://github.com/user-attachments/assets/43501886-138d-4999-9040-79bb86e424ea)

- 드롭아웃은 미니배치를 실행할 때마다 뉴런을 랜덤하게 잘라내서 새로운 모델을 생성하는 정규화 방법이다.

- 드롭아웃은 하나의 신경망 모델에서 무한히 많은 모델을 생성하는 배깅과 같다.

- 드롭아웃은 계산 시간이 거의 들지 않고 다양한 모델에 쉽게 적용할 수 있는 강력한 정규화 기법이다.

### 학습 단계

- 드롭아웃은 미니배치를 실행할 때마다 뉴런을 랜덤하게 잘라내서 매번 다른 모델을 생성한다.

![Image](https://github.com/user-attachments/assets/9e7dc74b-bf7d-4e33-a364-08a84fd5d272)

- 미니배치를 실행할 때마다 계층별로 뉴런의 이진 마스크를 생성한다.

- 이진 마스크는 뉴런별 드롭아웃 여부를 나타내며, 뉴런의 마스크값이 1이면 뉴런은 유지되고 마스크값이 0이면 드롭아웃 된다.

- 계층 l의 드롭아웃은 다음과 같이 정의한다.

$$ a^{(l)} = activation(W^{(l)T} x^{(l)} + b^{(l)}) $$
$$ r^{(l)} \sim \text{Bern}(p) $$
$$ \tilde{a}^{(l)} = a^{(l)} \odot r^{(l)} $$

### 추론 단계

- 추론 단계에서는 모든 뉴런이 활성화된다.

- 학습 시에 뉴런을 유지할 확률이 p였다면 계층 l의 추론은 다음과 같이 정의한다.
$$ a^{(l)} = activation(W^{(l)T} x^{(l)} + b^{(l)}) \times p $$

### 역드롭아웃

- 뉴런을 유지할 확률 p를 반드시 추론 시점에 곱해야 할까?

- 훈련 시점에 각 게층의 출력을 미리 p로 나누어 두면 원래의 추론 코드를 그대로 사용할 수 있다.

- 이런 아이디어를 적용한 방법이 역 드롭아웃이다.

- 다음 식과 같이 훈련 단계에서 각 계층 l의 출력을 p로 나누어주면 된다.

$$ a^{(l)} = activation(W^{(l)T} x^{(l)} + b^{(l)}) $$
$$ r^{(l)} \sim \text{Bern}(p) $$
$$ \tilde{a}^{(l)} = a^{(l)} \odot r^{(l)} / p $$

In [1]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import random

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# for reproducibility
random.seed(777)
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [3]:
# parameters
learning_rate = 0.001
training_epochs = 15
batch_size = 100
drop_prob = 0.3

In [4]:
# MNIST dataset
mnist_train = dsets.MNIST(root='MNIST_data/',
                          train=True,
                          transform=transforms.ToTensor(),
                          download=True)

mnist_test = dsets.MNIST(root='MNIST_data/',
                         train=False,
                         transform=transforms.ToTensor(),
                         download=True)

In [5]:
# dataset loader
data_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)

In [6]:
# nn layers
linear1 = torch.nn.Linear(784, 512, bias=True)
linear2 = torch.nn.Linear(512, 512, bias=True)
linear3 = torch.nn.Linear(512, 512, bias=True)
linear4 = torch.nn.Linear(512, 512, bias=True)
linear5 = torch.nn.Linear(512, 10, bias=True)
relu = torch.nn.ReLU()
dropout = torch.nn.Dropout(p=drop_prob)

In [7]:
# xavier initialization
torch.nn.init.xavier_uniform_(linear1.weight)
torch.nn.init.xavier_uniform_(linear2.weight)
torch.nn.init.xavier_uniform_(linear3.weight)
torch.nn.init.xavier_uniform_(linear4.weight)
torch.nn.init.xavier_uniform_(linear5.weight)

Parameter containing:
tensor([[-0.0565,  0.0423, -0.0155,  ...,  0.1012,  0.0459, -0.0191],
        [ 0.0772,  0.0452, -0.0638,  ...,  0.0476, -0.0638,  0.0528],
        [ 0.0311, -0.1023, -0.0701,  ...,  0.0412, -0.1004,  0.0738],
        ...,
        [ 0.0334,  0.0187, -0.1021,  ...,  0.0280, -0.0583, -0.1018],
        [-0.0506, -0.0939, -0.0467,  ..., -0.0554, -0.0325,  0.0640],
        [-0.0183, -0.0123,  0.1025,  ..., -0.0214,  0.0220, -0.0741]],
       requires_grad=True)

In [8]:
# model
model = torch.nn.Sequential(linear1, relu, dropout,
                            linear2, relu, dropout,
                            linear3, relu, dropout,
                            linear4, relu, dropout,
                            linear5).to(device)

In [9]:
# define cost/loss & optimizer
criterion = torch.nn.CrossEntropyLoss().to(device)    # Softmax is internally computed.
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [10]:
total_batch = len(data_loader)
model.train()    # set the model to train mode (dropout=True)
for epoch in range(training_epochs):
    avg_cost = 0

    for X, Y in data_loader:
        # reshape input image into [batch_size by 784]
        # label is not one-hot encoded
        X = X.view(-1, 28 * 28).to(device)
        Y = Y.to(device)

        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()

        avg_cost += cost.item()
        
    avg_cost /= total_batch

    print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost))

print('Learning finished')

Epoch: 0001 cost = 0.310847819
Epoch: 0002 cost = 0.143510714
Epoch: 0003 cost = 0.113777391
Epoch: 0004 cost = 0.094480618
Epoch: 0005 cost = 0.084084092
Epoch: 0006 cost = 0.074203303
Epoch: 0007 cost = 0.068857266
Epoch: 0008 cost = 0.064196247
Epoch: 0009 cost = 0.059554481
Epoch: 0010 cost = 0.057260883
Epoch: 0011 cost = 0.052823010
Epoch: 0012 cost = 0.052535421
Epoch: 0013 cost = 0.043824003
Epoch: 0014 cost = 0.047097244
Epoch: 0015 cost = 0.043089768
Learning finished


In [12]:
# Test model and check accuracy
with torch.no_grad():
    model.eval()    # set the model to evaluation mode (dropout=False)

    # Test the model using test sets
    X_test = mnist_test.test_data.view(-1, 28 * 28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)

    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy:', accuracy.item())

    # Get one and predict
    r = random.randint(0, len(mnist_test) - 1)
    X_single_data = mnist_test.test_data[r:r + 1].view(-1, 28 * 28).float().to(device)
    Y_single_data = mnist_test.test_labels[r:r + 1].to(device)

    print('Label: ', Y_single_data.item())
    single_prediction = model(X_single_data)
    print('Prediction: ', torch.argmax(single_prediction, 1).item())

Accuracy: 0.9745000004768372
Label:  5
Prediction:  5
